# data processing

## json create

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# package import

import pandas as pd
from data_pack import DataSetDefinition
import utils_cpu
import utils_gpu

from typing import List, Dict
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [4]:
dataset_def = DataSetDefinition(
    processed_dir_suffix="128_v2"
)

In [5]:
dataset_def.processed_data_root_dir

'./data/processed'

In [6]:
step2_split = dataset_def.step2_dict_generator(with_radiance_hint=True)

In [7]:
train_split = step2_split["train"]; test_split = step2_split["eval"]

In [8]:
train_split_df = pd.DataFrame(train_split); test_split_df = pd.DataFrame(test_split)

In [9]:
train_split_df

,image_id,viewpoint_id,lighting_condition_id,image_path,mask_path,output_dir,fov,mask_threshold,env_map,pls,use_gpu_for_rendering,resolution,hint_diffuse_path,hint_ggx0.05_path,hint_ggx0.13_path,hint_ggx0.34_path
0,obj_01_car,CC7,004,./data/processed/128_v2/train/images/obj_01_ca...,./data/processed/128_v2/train/masks/obj_01_car...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/004.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...
1,obj_01_car,CE2,007,./data/processed/128_v2/train/images/obj_01_ca...,./data/processed/128_v2/train/masks/obj_01_car...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/007.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...
2,obj_01_car,CB5,010,./data/processed/128_v2/train/images/obj_01_ca...,./data/processed/128_v2/train/masks/obj_01_car...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/010.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...
3,obj_01_car,CE2,008,./data/processed/128_v2/train/images/obj_01_ca...,./data/processed/128_v2/train/masks/obj_01_car...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/008.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...
4,obj_01_car,CE2,009,./data/processed/128_v2/train/images/obj_01_ca...,./data/processed/128_v2/train/masks/obj_01_car...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/009.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,obj_64_greenhead,CE2,010,./data/processed/128_v2/train/images/obj_64_gr...,./data/processed/128_v2/train/masks/obj_64_gre...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/010.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...
8188,obj_64_greenhead,CA2,001,./data/processed/128_v2/train/images/obj_64_gr...,./data/processed/128_v2/train/masks/obj_64_gre...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/001.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...
8189,obj_64_greenhead,NA7,001,./data/processed/128_v2/train/images/obj_64_gr...,./data/processed/128_v2/train/masks/obj_64_gre...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/001.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...
8190,obj_64_greenhead,NE1,013,./data/processed/128_v2/train/images/obj_64_gr...,./data/processed/128_v2/train/masks/obj_64_gre...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/013.hdr,"[[0, 0, 0]]",True,128,./da

In [10]:
# 같은 object, 같은 view_point에서 자기자신만 뺀 나머지의 light condition을 ref로서 한다.
# Step 1: Create the 'ref' column
def create_ref_column(df):
    # Group by 'image_id' and 'viewpoint_id'
    groups = df.groupby(['image_id', 'viewpoint_id'])
    refs = {}

    # Iterate over each group
    for (image_id, viewpoint_id), group in groups:
        # Create a list of image_paths and lighting_condition_ids for the group
        image_paths = group['image_path'].tolist()
        lighting_ids = group['lighting_condition_id'].tolist()

        # Iterate over each row in the group
        for idx, row in group.iterrows():
            # Exclude the current row's lighting_condition_id
            ref_paths = [
                p for lc_id, p in zip(lighting_ids, image_paths)
                if lc_id != row['lighting_condition_id']
            ]
            refs[idx] = ref_paths

    # Map the refs to the DataFrame
    df['ref'] = df.index.map(refs)
    return df

In [11]:
train_split_df[(train_split_df["viewpoint_id"] == "CF8")]

,image_id,viewpoint_id,lighting_condition_id,image_path,mask_path,output_dir,fov,mask_threshold,env_map,pls,use_gpu_for_rendering,resolution,hint_diffuse_path,hint_ggx0.05_path,hint_ggx0.13_path,hint_ggx0.34_path
6,obj_01_car,CF8,007,./data/processed/128_v2/train/images/obj_01_ca...,./data/processed/128_v2/train/masks/obj_01_car...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/007.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...
20,obj_01_car,CF8,004,./data/processed/128_v2/train/images/obj_01_ca...,./data/processed/128_v2/train/masks/obj_01_car...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/004.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...
26,obj_01_car,CF8,006,./data/processed/128_v2/train/images/obj_01_ca...,./data/processed/128_v2/train/masks/obj_01_car...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/006.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...
33,obj_01_car,CF8,013,./data/processed/128_v2/train/images/obj_01_ca...,./data/processed/128_v2/train/masks/obj_01_car...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/013.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...
35,obj_01_car,CF8,008,./data/processed/128_v2/train/images/obj_01_ca...,./data/processed/128_v2/train/masks/obj_01_car...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/008.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8153,obj_64_greenhead,CF8,001,./data/processed/128_v2/train/images/obj_64_gr...,./data/processed/128_v2/train/masks/obj_64_gre...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/001.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...
8158,obj_64_greenhead,CF8,004,./data/processed/128_v2/train/images/obj_64_gr...,./data/processed/128_v2/train/masks/obj_64_gre...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/004.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...
8162,obj_64_greenhead,CF8,005,./data/processed/128_v2/train/images/obj_64_gr...,./data/processed/128_v2/train/masks/obj_64_gre...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/005.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...,./data/processed/128_v2/train/hints/obj_64_gre...
8163,obj_64_greenhead,CF8,009,./data/processed/128_v2/train/images/obj_64_gr...,./data/processed/128_v2/train/masks/obj_64_gre...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/009.hdr,"[[0, 0, 0]]",True,128,

In [12]:
train_split_df_filtered = train_split_df[(train_split_df["viewpoint_id"] == "CF8") & (train_split_df["lighting_condition_id"] =="011")]
train_split_df_filtered

,image_id,viewpoint_id,lighting_condition_id,image_path,mask_path,output_dir,fov,mask_threshold,env_map,pls,use_gpu_for_rendering,resolution,hint_diffuse_path,hint_ggx0.05_path,hint_ggx0.13_path,hint_ggx0.34_path
53,obj_01_car,CF8,011,./data/processed/128_v2/train/images/obj_01_ca...,./data/processed/128_v2/train/masks/obj_01_car...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...,./data/processed/128_v2/train/hints/obj_01_car...
223,obj_02_egg,CF8,011,./data/processed/128_v2/train/images/obj_02_eg...,./data/processed/128_v2/train/masks/obj_02_egg...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_02_egg...,./data/processed/128_v2/train/hints/obj_02_egg...,./data/processed/128_v2/train/hints/obj_02_egg...,./data/processed/128_v2/train/hints/obj_02_egg...
288,obj_03_chicken,CF8,011,./data/processed/128_v2/train/images/obj_03_ch...,./data/processed/128_v2/train/masks/obj_03_chi...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_03_chi...,./data/processed/128_v2/train/hints/obj_03_chi...,./data/processed/128_v2/train/hints/obj_03_chi...,./data/processed/128_v2/train/hints/obj_03_chi...
396,obj_04_stone,CF8,011,./data/processed/128_v2/train/images/obj_04_st...,./data/processed/128_v2/train/masks/obj_04_sto...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_04_sto...,./data/processed/128_v2/train/hints/obj_04_sto...,./data/processed/128_v2/train/hints/obj_04_sto...,./data/processed/128_v2/train/hints/obj_04_sto...
579,obj_05_bird,CF8,011,./data/processed/128_v2/train/images/obj_05_bi...,./data/processed/128_v2/train/masks/obj_05_bir...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_05_bir...,./data/processed/128_v2/train/hints/obj_05_bir...,./data/processed/128_v2/train/hints/obj_05_bir...,./data/processed/128_v2/train/hints/obj_05_bir...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,obj_60-fabric-green-hedgehog_011,CF8,011,./data/processed/128_v2/train/images/obj_60-fa...,./data/processed/128_v2/train/masks/obj_60-fab...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_60-fab...,./data/processed/128_v2/train/hints/obj_60-fab...,./data/processed/128_v2/train/hints/obj_60-fab...,./data/processed/128_v2/train/hints/obj_60-fab...
7763,obj_61-fabric-dolphin_011,CF8,011,./data/processed/128_v2/train/images/obj_61-fa...,./data/processed/128_v2/train/masks/obj_61-fab...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_61-fab...,./data/processed/128_v2/train/hints/obj_61-fab...,./data/processed/128_v2/train/hints/obj_61-fab...,./data/processed/128_v2/train/hints/obj_61-fab...
7920,obj_62-fabric-birthday-cake_011,CF8,011,./data/processed/128_v2/train/images/obj_62-fa...,./data/processed/128_v2/train/masks/obj_62-fab...,./data/processed/128_v2/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v2/train/hints/obj_62-fab...,./data/processed/128_v2/train/hints/obj_62-fab...,./data/processed/128_v2/train/hints/obj_62-fab...,./data/processed/128_v2/train/hints/obj_62-fab...
7940,obj_63-fabric-friends-cup_011,CF8,011,./data/processed/128_v2/train/images/obj_63-fa...,./data/processed/128_v2/train/masks/obj_63-fab...,./data/processed/128_v2/train/hints,None,0.25

In [13]:
# directory structure -> list of data

step1_datadict_train_list: List[Dict[str, str]] = []
step1_datadict_eval_list: List[Dict[str, str]] = []

for each_class_name in dataset_def.class_name_list:
    for each_view_point in dataset_def.view_point_list:
        for each_light_condition in dataset_def.light_condition_list:

            image_path = DataSetDefinition.get_image_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                light_condition=each_light_condition,
                view_point=each_view_point,
            )
            mask_path = DataSetDefinition.get_mask_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                view_point=each_view_point,
            )

            parsed_name = DataSetDefinition.get_image_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                light_condition=each_light_condition,
                view_point=each_view_point,
                parsed=True,
            )

            if parsed_name in dataset_def.eval_split:
                step1_datadict_eval_list.append(
                    {
                        "key": f"{each_class_name}_{each_light_condition}_{each_view_point}",
                        "image_path": image_path,
                        "mask_path": mask_path,
                    }
                )
            else:
                step1_datadict_train_list.append(
                    {
                        "key": f"{each_class_name}_{each_light_condition}_          {each_view_point}",
                        "image_path": image_path,
                        "mask_path": mask_path,
                    }
                )

# caption generation

In [14]:
# 여기서 dict로 for문 돌려버리자.
# 64개의 object에 대해서 가장 광량이 높은 조건으로 가져오도록 한다.
# obj_01_car_013_CF8.png
captioner = utils_gpu.BLIPI2T()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
priority_mapping = {'013': 1, '012': 2}
train_split_df['priority'] = train_split_df['lighting_condition_id'].map(priority_mapping).fillna(3)
train_split_df_sorted = train_split_df.sort_values(by=['image_id', 'viewpoint_id', 'priority'])
df_selected = train_split_df_sorted.drop_duplicates(subset=['image_id', 'viewpoint_id'], keep='first')
for index, row in df_selected.iterrows():
    caption = captioner(row['image_path'])
    df_selected.at[index, 'caption'] = caption

RuntimeError: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]

In [ ]:
captioner(row['image_path'])

RuntimeError: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]

In [ ]:
# Step 2: Add the 'caption' column from 'caption_dict'
# Assuming 'caption_dict' is a dictionary with 'image_id' as keys and 'caption' as values
df['caption'] = df['image_id'].map(caption_dict)